In [ ]:
import pandas as pd
from IPython.display import display, HTML
import numpy as np

In [ ]:
models = {
    'CF': '/mnt/data/vikuen/data/daily-mail/results/evaluation_collaborative.csv',
    'CF22': '/mnt/data/vikuen/data/daily-mail/results/evaluation_collaborative_22.csv',
    'TF-IDF': '/mnt/data/vikuen/data/daily-mail/results/evaluation_bow.csv',
    'BRF': '/mnt/data/vikuen/data/daily-mail/results/evaluation_data_fusion.csv',
    'node2vec': '/mnt/data/vikuen/data/daily-mail/results/evaluation_data_node2vec_update.csv',
    'DeepCoNN': '/mnt/data/vikuen/data/daily-mail/results/evaluation_data_deepconn_fm.csv',    
    'HyCoNN': '/mnt/data/vikuen/data/daily-mail/results/evaluation_data_main_model_update.csv',
    'NDRF': '/mnt/data/vikuen/data/daily-mail/results/evaluation_data_fusion_node2vec_deepconn.csv',
}

In [ ]:
K = [1, 3, 5, 10, 15]

In [ ]:
def evaluate_model(df):
    recalls = []
    precisions = []
    f1 = []
    MAP = []
    for k in K:
        selected = df[df.k == k] 
        global_recall_at_k = selected.hits_at_k.sum() / float(selected.interacted_count.sum())
        recall = selected.recall.mean()
        recalls.append(recall)
        precision = selected.precision.mean()
        precisions.append(precision)
        f1.append((2*(recall * precision)) / (recall + precision))
        MAP.append(selected.AP.mean())
    
    metrics = {
        'precision': {},
        'f1': {},
        'MAP': {},
        'recall': {}
    }
    for i in range(len(K)):
        metrics['recall'][f'recall@{K[i]}'] = recalls[i]
        metrics['precision'][f'precision@{K[i]}'] = precisions[i]    
        metrics['f1'][f'f1@{K[i]}'] = f1[i]    
        metrics['MAP'][f'MAP@{K[i]}'] = MAP[i]
    return metrics

In [ ]:
metrics = ['recall', 'precision', 'f1', 'MAP']
col_names = ['author_id', 'k', 'hits_at_k', 'interacted_count', 'precision', 'recall', 'AP', 'documents']
eval_results = []
df = None
dfs = []
for model, path in models.items():
    print(path)
    df = pd.read_csv(path)
    eval_results.append(evaluate_model(df))

In [ ]:
out_dfs = []
for metric in metrics:
    out_dict = {}
    out_dict['metric'] = []
    for model in models:
        out_dict[model] = []
    
    for k in K:
        out_dict['metric'].append(f'{metric}@{k}')
    
    for i, model in enumerate(models):
        results = eval_results[i][metric]
        for _, value in results.items():
            out_dict[model].append(value)
            
    out_df = pd.DataFrame(out_dict)
    out_df = out_df.round(3)
    display(HTML(out_df.to_html()))
    print(out_df.to_latex())
    out_df.to_csv(f'{metric}.csv', index=False)
    out_dfs.append(out_df)

In [ ]:
%matplotlib inline

In [ ]:
import seaborn as sns

In [ ]:
df_recall = out_dfs[0]

In [ ]:
df_recall['recall'] = df_recall['metric'].apply(lambda x: int(x.replace('recall@', '')))

In [ ]:
df_recall = df_recall.drop(columns=['metric'])

In [ ]:
df_recall.index = df_recall.recall

In [ ]:
to_plot = df_recall.transpose().stack().reset_index()
to_plot = to_plot[to_plot['level_0'] != 'recall']

In [ ]:
to_plot['k'] = to_plot['recall']
to_plot['Recall'] = to_plot[0]
to_plot = to_plot[to_plot['level_0'].isin( ['CF','node2vec', 'BRF', 'HyCoNN', 'DeepCoNN', 'NDRF', 'TF-IDF'])]
# ['CF','node2vec', 'BRF', 'HyCoNN', 'DeepCoNN', 'NDRF', 'TF-IDF']
to_plot['Model'] = to_plot['level_0']

In [ ]:
sns.set()
sns.set_context('talk', font_scale=1.8)

In [ ]:
 import matplotlib.pyplot as plt

In [ ]:
f, ax = plt.subplots(figsize=(9,6)) 
filled_markers = ('o', 'v', '^', '<', '>', '8', 's', 'p', '*', 'h', 'H', 'D', 'd', 'P', 'X')
dash_styles = ["",
               (4, 1.5),
               (1, 1),
               (3, 1, 1.5, 1),
               (5, 1, 1, 1),
               (5, 1, 2, 1, 2, 1),
               (2, 2, 3, 1.5),
               (1, 2.5, 3, 1.2)]
ax = sns.lineplot(x='k', y='Recall', hue='Model',style="Model", data=to_plot[to_plot['level_0'] != 'recall'],
                  ax=ax, linewidth=3, dashes=dash_styles, markersize=8)
plt.setp(ax,xticks=K) 
leg = plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., frameon=False)

# set the linewidth of each legend object
for legobj in leg.legendHandles:
    legobj.set_linewidth(3.0)
save_to_pdf('./plots/results_daily-mail.pdf', ax)

In [ ]:
def save_to_pdf(filename, ax):
    fig = ax.get_figure()
    fig.savefig(filename, bbox_inches='tight')